Installing environment variables

In [ ]:
!pip install langchain
!pip install langchain_experimental
!pip install google-generativeai
!pip install langchain-google-genai  #To access in-built Google AI Gemini models
!pip install --upgrade --quiet  langchain-google-genai # to upgrade to newer version for 'key configuration'
!pip install -U google-generativeai  # To install Python SDK package for Gemini API & key configuration
!pip install -U langchain-community
!pip install pandas
!pip install NumPy
!pip install google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.9/378.9 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.8/275.8 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.4 MB/s eta 0:00:00


2: Get API key from google-gemini site and configure to use for project as below:

In [ ]:
import os
import pandas as pd
import numpy as np
from langchain_google_genai import ChatGoogleGenerativeAI as cgenai # To query project CSV
from langchain_experimental.agents import create_csv_agent
from langchain_community.document_loaders.csv_loader import CSVLoader
from google.colab import userdata
from langchain_google_genai import GoogleGenerativeAI as genai # To use for Google AI Studio codes
from langchain_community.chat_models import ChatGooglePalm
from langchain_core.messages import HumanMessage, SystemMessage # To enable adding system message to gemini
from google.auth import default             # importing the google.auth library


# Get the API key from Google AI studio

# Retrieve your Google API key
GOOGLE_KEY = userdata.get('GOOGLE_KEY') #use getpass() if userdata() doesn't work

#Authenticate and set the API key
creds,_ = default()  # default() to get credentials
creds.token= GOOGLE_KEY ## sets the API key

#setting key API as environment variable
os.environ['GOOGLE_KEY'] = 'GOOGLE_KEY' # Ensure to replace with your actual key


3: Loading the CSV file

In [2]:
# Read in csv
df=pd.read_csv('hobbyList.csv')

NameError: name 'pd' is not defined

In [ ]:
# Checking the CSV uploaded correctely

df.head()

,Hobby-name,Type
0,Air sports,Outdoors and sports
1,Airsoft,Outdoors and sports
2,Amateur geology,Outdoors and sports
3,Amusement park visiting,Outdoors and sports
4,Archery,Outdoors and sports


4: Few Data Exploration Analysis

In [ ]:
# Checking dataset size

df.shape

(423, 2)

In [ ]:
# Most Hobby
df.mode()

,Hobby-name,Type
0,Radio-controlled model playing,Outdoors and sports
1,Volleyball,NaN


In [1]:
# Checking for missing values
#df.isnull().sum()
df['Type']== np.nan

NameError: name 'df' is not defined

In [ ]:
# Frequency table to know how many category levels exist and
# number of hobbies in each level

hobbyType= pd.crosstab(index=df['Type'], columns='count')
hobbyType

col_0,count
Type,
Educational hobby,39
Indoor Collection hobby,45
Indoor Competitive hobby,63
Indoor Observation hobby,9
Outdoor Collection hobby,13
Outdoor Competitive hobby,70
Outdoor Observation hobby,17
Outdoors and sports,167


In [ ]:
# Number of hobbies duplicated accross the type categories
df['Hobby-name'].duplicated().sum()

39

In [ ]:
# Name of the 39 duplicated hobbies
df[df['Hobby-name'].duplicated()]

,Hobby-name,Type
169,Astronomy,Educational hobby
257,Magnet fishing,Outdoor Collection hobby
258,Metal detecting,Outdoor Collection hobby
268,Badminton,Indoor Competitive hobby
287,Darts,Indoor Competitive hobby
295,Gymnastics,Indoor Competitive hobby
306,Martial arts,Indoor Competitive hobby
313,Radio-controlled model playing,Indoor Competitive hobby
322,Volleyball,Indoor Competitive hobby
327,Airsoft,Outdoor Competitive hobby


5: Setting model to acces in-built google ai model for answers to prompt questions


a: Accessing the in-built model -Gemini pro

In [ ]:
# Define the model as llm2 with 'gemini-pro as the model to use

# set environment for API key again
os.environ['GOOGLE_API_KEY'] = 'GOOGLE_API_KEY' # use your actual API key

llm2 = cgenai(model="gemini-pro")



In [ ]:
## Outputs from prompt questions for the llm2 model

result = llm2.invoke("What hobby types are available?")
print(result.content)

**Creative Hobbies:**

* **Art:** Painting, drawing, sculpting, photography, graphic design
* **Music:** Playing instruments, singing, songwriting, producing
* **Writing:** Creative writing, journaling, poetry, blogging
* **Crafts:** Knitting, crochet, sewing, woodworking, jewelry making
* **Gardening:** Growing plants, landscaping, floral design

**Intellectual Hobbies:**

* **Reading:** Fiction, non-fiction, poetry, philosophy
* **Learning new skills:** Languages, coding, computer science, history
* **Puzzles:** Crosswords, Sudoku, jigsaw puzzles, puzzles
* **Board games:** Chess, checkers, strategy games
* **Collecting:** Stamps, coins, books, antiques

**Active Hobbies:**

* **Sports:** Team sports (e.g., basketball, soccer), individual sports (e.g., running, swimming)
* **Fitness:** Yoga, Pilates, Zumba, weightlifting
* **Outdoor activities:** Hiking, camping, kayaking, skiing
* **Martial arts:** Karate, tae kwon do, judo
* **Dancing:** Ballroom dancing, hip-hop, ballet

**Social 

In [ ]:
result = llm2.invoke("What do you recommend for challenging but indoortype hobbies?")
print(result.content)

**Intellectual Challenges:**

* **Chess or Go:** Strategy games that require critical thinking, problem-solving, and anticipation.
* **Puzzles and Riddles:** Engage in brain teasers, crosswords, Sudoku, or escape rooms to test your deductive reasoning and logical skills.
* **Coding and Programming:** Learn a programming language and create your own software, apps, or games, fostering creativity and problem-solving.
* **Creative Writing:** Dive into storytelling, poetry, or journaling to express yourself, explore your imagination, and improve your writing abilities.
* **Learning a New Language:** Expand your horizons by mastering a foreign language through apps, online courses, or language exchange platforms.

**Creative and Artistic Pursuits:**

* **Painting or Drawing:** Bring your imagination to life on canvas or paper with brushes, pencils, and paints.
* **Photography:** Capture the world around you through the lens of a camera, experimenting with composition, lighting, and editing 

In [ ]:
result = llm2.invoke("I am bored, what can I do to pass the time?")
print(result.content)

**Creative Activities:**

* **Draw, paint, or sketch:** Express your creativity and explore different mediums.
* **Write a story, poem, or song:** Let your imagination run wild and craft a unique piece of writing.
* **Dance or sing:** Move your body and express yourself through rhythm and melody.
* **Learn a new instrument:** Challenge yourself by mastering a musical instrument.
* **Try photography:** Capture the beauty of the world around you through the lens of a camera.

**Intellectual Activities:**

* **Read a book or listen to an audiobook:** Dive into a captivating story, expand your knowledge, or learn new skills.
* **Solve puzzles or play brain games:** Exercise your mind and improve your problem-solving abilities.
* **Learn a new language:** Expand your horizons by immersing yourself in a different culture and language.
* **Take an online course:** Enhance your skills or learn something new in a convenient and accessible way.
* **Do research on a topic that interests you:** Ex

b:  In-built model tunning:

Add system message to the 1st human message by setting 'convert_system_message_to_human' = True, since Gemini does not support system message at the moment. (NB:This will soon be deprecated)

In [ ]:
# Adding system message to google in-built model - Gemini.

model = cgenai(model="gemini-pro", convert_system_message_to_human=True)
messages=[
        SystemMessage(content="You are an hobby expert, your name is Cason. Be polite at all times. \
                              Answer correctly as best as you can."),
        HumanMessage(content="What can I do indoors but also active?"),
    ]
print(model.invoke(messages).content)

/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:350: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Hello there! My name is Cason and I'm an expert in hobbies. I'd be happy to help you find an indoor activity that is also active. Here are a few ideas:

1. Indoor rock climbing: This is a great way to get a full-body workout while having fun. There are many indoor rock climbing gyms that offer classes and rentals for beginners.

2. Trampoline parks: These parks are filled with trampolines, foam pits, and other obstacles that you can jump and play on. They're a great way to get some cardio and have a blast.

3. Indoor swimming: Swimming is a great low-impact way to get exercise. Many community centers and fitness clubs have indoor pools that you can use.

4. Dance classes: There are many different types of dance classes available, such as Zumba, hip-hop, and ballet. Dance is a great way to get moving and have fun.

5. Martial arts: Martial arts classes are a great way to learn self-defense and get a workout. There are many different types of martial arts to choose from, such as karate, 

6: Modeling for RAG:

Using Agent:
Accessing the project's CSV- Hobbylist dataset and generating responses to prompts concerning the CSV & gemini pro.

In [ ]:
# Retrieve your Google API key
GOOGLE_API_KEY = userdata.get('GOOGLE_KEY')

#Authenticate and set the API key
creds,_ = default()  # default() to get credentials
creds.token= GOOGLE_KEY ## sets the API key

#setting key API as environment variable
os.environ['GOOGLE_API_KEY'] = 'GOOGLE_API_KEY'

# Create the agent
agent = create_csv_agent(cgenai(temperature=0, model='gemini-pro'),'hobbyList.csv',
                         verbose=True, allow_dangerous_code=True)

# Run the agent.
result1 = agent.run("[Which 10 hobby names are duplicated in df[df['Hobby-name']]]?")
# 10 requested to avoid overload
print(result1)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Thought: I should use the `duplicated` function to find the duplicated values in the `Hobby-name` column.
Action: python_repl_ast
Action Input: df[df['Hobby-name'].duplicated(keep=False)].head(10)      Hobby-name                 Type
1        Airsoft  Outdoors and sports
4        Archery  Outdoors and sports
7      Astronomy  Outdoors and sports
9      Badminton  Outdoors and sports
11      Baseball  Outdoors and sports
14    Beekeeping  Outdoors and sports
15  Birdwatching  Outdoors and sports
29      Climbing  Outdoors and sports
31       Croquet  Outdoors and sports
32       Cycling  Outdoors and sportsFinal Answer: Airsoft, Archery, Astronomy, Badminton, Baseball, Beekeeping, Birdwatching, Climbing, Croquet, Cycling

> Finished chain.
Airsoft, Archery, Astronomy, Badminton, Baseball, Beekeeping, Birdwatching, Climbing, Croquet, Cycling


In [ ]:
result1 = agent.run("[What category levels exists in the csv?]")
print(result1)



> Entering new AgentExecutor chain...
Thought: I should use the `unique` function to find the unique values in the `Type` column.
Action: python_repl_ast
Action Input: df['Type'].unique()['Outdoors and sports' 'Educational hobby' 'Indoor Collection hobby'
 'Outdoor Collection hobby' 'Indoor Competitive hobby'
 'Outdoor Competitive hobby' 'Indoor Observation hobby'
 'Outdoor Observation hobby']I now know the final answer
Final Answer: ['Outdoors and sports' 'Educational hobby' 'Indoor Collection hobby'
 'Outdoor Collection hobby' 'Indoor Competitive hobby'
 'Outdoor Competitive hobby' 'Indoor Observation hobby'
 'Outdoor Observation hobby']

> Finished chain.
['Outdoors and sports' 'Educational hobby' 'Indoor Collection hobby'
 'Outdoor Collection hobby' 'Indoor Competitive hobby'
 'Outdoor Competitive hobby' 'Indoor Observation hobby'
 'Outdoor Observation hobby']


In [ ]:
result1 = agent.run("[I am 65 years old, I don't like the sun and challenges,what hobbies can you recommend?]")
print(result1)



> Entering new AgentExecutor chain...
Thought: I should filter the dataframe to only include hobbies that are not outdoors and sports
Action: python_repl_ast
Action Input: df[df['Type'] != 'Outdoors and sports']             Hobby-name                       Type
167         Archaeology          Educational hobby
168        Architecture          Educational hobby
169           Astronomy          Educational hobby
170           Animation          Educational hobby
171           Aerospace          Educational hobby
..                  ...                        ...
418          Geocaching  Outdoor Observation hobby
419         Gongoozling  Outdoor Observation hobby
420             Herping  Outdoor Observation hobby
421  Hiking/backpacking  Outdoor Observation hobby
422         Meteorology  Outdoor Observation hobby

[256 rows x 2 columns]Final Answer: I recommend Archaeology, Architecture, Astronomy, Animation, Aerospace, Birdwatching, Board games, Book collecting, Card games, Chess, Col

7:Accessing the RAG-tuned model (hobbylist csv + gemini pro) and generating responses to all prompts all. Here ChatGooglePalm command is used to load the csv

In [ ]:
# Loading the hobbylist CSV into the session

loader = CSVLoader(file_path='hobbyList.csv')
data_hobby =  pd.DataFrame(loader.load())

In [ ]:
 # Initialize the model with API key and use 'ChatGooglePalm' to access the project dataset
Chat_model = ChatGooglePalm(model=['gemini-pro','data_hobby'], google_api_key=os.environ['GOOGLE_API_KEY'],
                            temperature=0.2)

In [ ]:
result2 = Chat_model.invoke("What hobbies are duplicated in data_hobby['Hobby-name']?")
print(result2.content)


The following hobbies are duplicated in hobbylist.csv['Hobby-name']:

* Reading
* Writing
* Cooking
* Dancing
* Singing
* Playing an instrument
* Playing sports
* Traveling
* Hiking
* Camping
* Fishing
* Swimming
* Biking
* Running
* Yoga
* Meditation
* Gardening
* Volunteering
* Collecting
* Playing video games
* Watching movies
* Watching TV
* Listening to music
* Playing board games
* Playing card games
* Going to the movies
* Going to the park
* Going to the zoo
* Going to the museum
* Going to the library
* Going to the beach
* Going to the mountains
* Going to the lake
* Going to the river
* Going to the forest
* Going to the desert
* Going to the city
* Going to the country
* Going to the world

I hope this is helpful!


In [ ]:
result2 = Chat_model.invoke("How many hobbies are duplicated in the data_hobby['Hobby-name']?")
print(result2.content)

There are 10 duplicated hobbies in the hobbylist.csv['Hobby-name']. The duplicated hobbies are:

* Reading
* Writing
* Playing video games
* Listening to music
* Watching movies
* Traveling
* Cooking
* Hiking
* Biking
* Swimming

I found the duplicated hobbies by first sorting the hobbylist.csv['Hobby-name'] column in alphabetical order. This gave me a list of hobbies in alphabetical order. I then used a Python program to compare each hobby in the list to the hobbies that came before it. If a hobby was found to be the same as a hobby that came before it, it was added to a list of duplicated hobbies.

The Python program I used is as follows:

```python
import pandas as pd

df = pd.read_csv('hobbylist.csv')

duplicate_hobbies = []

for i in range(len(df['Hobby-name'])):
    for j in range(i + 1, len(df['Hobby-name'])):
        if df['Hobby-name'][i] == df['Hobby-name'][j]:
            duplicate_hobbies.append(df['Hobby-name'][i])

print(duplicate_hobbies)
```

The output of the Python pr

In [ ]:
result2 = Chat_model.invoke("[What hobby types are in Outdoors and Sports?]")
print(result2.content)


There are many different types of hobbies that fall under the categories of outdoors and sports. Some popular outdoor hobbies include hiking, camping, fishing, hunting, and biking. Some popular sports hobbies include basketball, football, soccer, baseball, and tennis.

Outdoor hobbies are activities that are typically done in nature or in a natural setting. They can be physical activities, such as hiking or biking, or they can be more passive activities, such as fishing or camping. Outdoor hobbies can be a great way to get exercise, enjoy the outdoors, and connect with nature.

Sports hobbies are activities that are typically done in a competitive setting. They can be individual sports, such as tennis or golf, or they can be team sports, such as basketball or football. Sports hobbies can be a great way to stay active, have fun, and meet new people.

Here are some more specific examples of outdoor and sports hobbies:

* **Outdoor hobbies:**
    * Hiking: Hiking is a great way to get exe

In [ ]:
result2 = Chat_model.invoke("I don't like sun, I don't like challenge, I am 65 years old, what hobbies would you recommend?")
print(result2.content)


Here are some hobbies that you might enjoy, given your preferences:

* Reading: Reading is a great way to relax and escape from the world for a while. You can read fiction, non-fiction, or anything in between. There are books on just about every topic imaginable, so you're sure to find something that interests you.
* Writing: If you enjoy writing, you could start a blog, write short stories or poems, or even write a book. Writing can be a great way to express yourself and share your thoughts and ideas with others.
* Gardening: Gardening is a great way to get some exercise and fresh air, and it can also be very rewarding to see your plants grow and thrive. If you don't have a lot of space, you can even start a small indoor garden.
* Cooking: Cooking is a great way to be creative and experiment with new flavors. You can also learn about different cultures by trying new recipes.
* Knitting or crocheting: Knitting and crocheting are both great hobbies that can be relaxing and meditative. Y